## imports & functions

In [1]:
PATH_TR = 'uebung6.train'


In [19]:
import numpy as np
import pandas as pd
import re
import sklearn_crfsuite
import pickle

In [35]:
from sklearn_crfsuite import metrics

In [ ]:


from itertools import chain

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
#from sklearn.cross_validation import cross_val_score
#from sklearn.grid_search import RandomizedSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [14]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

## train model

In [15]:
with open(PATH_TR, 'r') as f:
    lines = f.readlines()
    
lines = [line.strip() for line in lines]
token = ['<START>' if str.split(line, ' ')[0] == '' else str.split(line, ' ')[0] for line in lines]
tag = ['<START>' if str.split(line, ' ')[0] == '' else str.split(line, ' ')[1] for line in lines]
label = ['<START>' if str.split(line, ' ')[0] == '' else str.split(line, ' ')[2] for line in lines]

In [16]:
def to_str_token(token, word = True):
  token_seq = " ".join(token)
  
  if(word):
    token_seq = re.sub('-DOCSTART- <START>', '',token_seq)
  
  else:
    token_seq = re.sub('X <START>', '',token_seq)
  
  sentence_tok = token_seq.split("<START>")
  sentence_tok = [s.strip() for s in sentence_tok]
  sentence_tok = [s.split(' ') for s in sentence_tok]
  sentence_list =  [list(filter(None, s)) for s in sentence_tok]
  sentence_list = list(filter(None, sentence_tok))
  
  return(sentence_tok)

In [17]:
sent_tok = to_str_token(token)[:-1]
sent_tag = to_str_token(tag, False)[:-1]
sent_lab = to_str_token(label, False)[:-1]


len(sent_tok), len(sent_tag), len(sent_lab)

(12000, 12000, 12000)

In [18]:
train_sents = []

for i in range(len(sent_tok)):
  s = []
  for t,p,y in zip(sent_tok[i], sent_tag[i], sent_lab[i]):
    s.append((t,p,y))
  train_sents.append(s)

In [24]:
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

In [25]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CPU times: user 25.2 s, sys: 380 ms, total: 25.5 s
Wall time: 25.8 s


In [28]:
model_file = "crf_model.p"
pickle.dump(crf, open(model_file, "wb"))

# Make predictions

In [30]:
PATH_VAL = 'uebung6.val'
model_file = "crf_model.p"

with open(PATH_VAL, 'r') as f:
    lines = f.readlines()
    
lines = [line.strip() for line in lines]
token = ['<START>' if str.split(line, ' ')[0] == '' else str.split(line, ' ')[0] for line in lines]
tag = ['<START>' if str.split(line, ' ')[0] == '' else str.split(line, ' ')[1] for line in lines]
label = ['<START>' if str.split(line, ' ')[0] == '' else str.split(line, ' ')[2] for line in lines]

sent_tok = to_str_token(token)[:-1]
sent_tag = to_str_token(tag, False)[:-1]
sent_lab = to_str_token(label, False)[:-1]


len(sent_tok), len(sent_tag), len(sent_lab)


val_sents = []

for i in range(len(sent_tok)):
  s = []
  for t,p,y in zip(sent_tok[i], sent_tag[i], sent_lab[i]):
    s.append((t,p,y))
  val_sents.append(s)


X_val = [sent2features(s) for s in val_sents]
y_val = [sent2labels(s) for s in val_sents]


In [29]:
labels = list(crf.classes_)

In [31]:
crf =  pickle.load(open(model_file, 'rb'))

In [36]:
y_pred = crf.predict(X_val)
metrics.flat_f1_score(y_val, y_pred,
                      average='weighted', labels=labels)

0.9591408698962599

In [37]:
y_pred = np.concatenate(y_pred)

In [38]:
output_lines = []
j = 0 

for i in range(len(token)):
  if token[i] == '-DOCSTART-':
    output_lines.append('-DOCSTART- X X O')
  elif token[i] == '<START>':
    output_lines.append('')
  else:
    txt = token[i] + " " + tag[i] + " " + y_pred[j]
    j += 1
    output_lines.append(txt)    

In [39]:
len(output_lines), len(lines)

(56246, 56246)

In [40]:
OUTPUT_FILE = "output_file.iob"
#OUTPUT_FILE = "output_file.iob"
with open(OUTPUT_FILE,mode = 'w') as f: 
    f.write("\n".join(output_lines))